# Myelin Mapping Notebook

These workflows process T1 and T2 weighted images to then be used to quantify cortical myelin content per methods described in Gordon et al., 2017 (doi: 10.1016/j.neuron.2017.07.011)

In [1]:
# Import all othe functions/nodes being used
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.interfaces.fsl import FSLCommand, BET, Reorient2Std
from nipype.interfaces.freesurfer import BBRegister
from nipype.interfaces.ants import N4BiasFieldCorrection
import networkx
import graphviz

# Declare study-specific variables--Jay will change to match the Mac
study_home = '/Users/dendrite/Desktop/Jay'
raw_data = study_home + '/raw'
output_dir = study_home + '/proc'
workflow_dir = study_home + '/workflows'
fs_dir = study_home + '/freesurfer'

subject_ID = '3000'

# Set any processing settings
FSLCommand.set_default_output_type('NIFTI_GZ')

In [2]:
# datasink node
datasink = Node(DataSink(base_directory=output_dir), name='datasink')

## Preprocess T2w data
The nodes and workflow below preprocess T2-weighted images to match T1-weighted data processed externally using FreeSurfer v6.

In [3]:
# Selectfiles node to grab subject T2w data
t2w_templates = {'anat': raw_data + '/t2w_2.nii.gz'}
t2w_source = Node(SelectFiles(t2w_templates), name='t2w_source')

# skullstrip the T2w using BET (FSL)
t2strip = Node(BET(), name='t2strip')

# N4 bias correction of the T2w data (ANTs)
bias_correct = Node(N4BiasFieldCorrection(),name='bias_correct')

# Reorient T2w to standard
reorient_t2w = Node(Reorient2Std(), name='reorient_t2w')

# Register T2w to T1w using FreeSurfer's Boundary-Based Registration
register = Node(BBRegister(subject_id = subject_ID, subjects_dir = fs_dir, contrast_type='t2'), name='register')


In [4]:
# Write the workflow
preprocess_t2 = Workflow(name='preprocess_t2')
preprocess_t2.connect([(t2w_source, t2strip,[('anat','in_file')]),
                       (t2strip, reorient_t2w,[('out_file','in_file')]),
                       (reorient_t2w, bias_correct,[('out_file','input_image')]),
                       (bias_correct, register,[('output_image','source_file')]),
                       (t2strip, datasink,[('out_file','skullstripped_t2w')]),
                       (register, datasink,[('registered_file','registered_t2w')])
                       ])

preprocess_t2.base_dir = workflow_dir
preprocess_t2.write_graph(graph2use='flat')
preprocess_t2.run('MultiProc', plugin_args={'n_procs': 2, 'memory_gb': 10})

191014-10:06:20,62 workflow INFO:
	 Generated workflow graph: /Users/dendrite/Desktop/Jay/workflows/preprocess_t2/graph.png (graph2use=flat, simple_form=True).
191014-10:06:20,91 workflow INFO:
	 Workflow preprocess_t2 settings: ['check', 'execution', 'logging', 'monitoring']
191014-10:06:20,100 workflow INFO:
	 Running in parallel.
191014-10:06:20,104 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 2/2.
191014-10:06:20,147 workflow INFO:
	 [Node] Setting-up "preprocess_t2.t2w_source" in "/Users/dendrite/Desktop/Jay/workflows/preprocess_t2/t2w_source".
191014-10:06:20,158 workflow INFO:
	 [Node] Running "t2w_source" ("nipype.interfaces.io.SelectFiles")
191014-10:06:20,167 workflow INFO:
	 [Node] Finished "preprocess_t2.t2w_source".
191014-10:06:22,104 workflow INFO:
	 [Job 0] Completed (preprocess_t2.t2w_source).
191014-10:06:22,106 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 10.00/10.

191014-10:08:00,180 workflow INFO:
	 [MultiProc] Running 0 tasks, and 0 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 2/2.
191014-10:08:02,174 workflow INFO:
	 ***********************************
191014-10:08:02,175 workflow ERROR:
	 could not run node: preprocess_t2.register
191014-10:08:02,176 workflow INFO:
	 crashfile: /Users/dendrite/Desktop/Jay/5yop/crash-20191014-100800-dendrite-register-82288422-3971-4be4-b9da-435b0307a8aa.pklz
191014-10:08:02,176 workflow INFO:
	 ***********************************


RuntimeError: Workflow did not execute cleanly. Check log for details

In [ ]:
# FreeSurferSource node to grab processed T1w data from freesurfer
t1w_source = Node(FreeSurferSource(subject_dir=fs_dir, subject_id=subject_ID), name='t1w_source')
 
# Reorient T1w to standard (FSL reorient2std)
reorient_t1w = Node(Reorient2Std(), name='reorient_t1w')